# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [7]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [8]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [9]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [10]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: AI-powered tools can generate high-quality content such as blog posts, social media posts, product descriptions, and even entire articles. This saves time and resources for content creators.
2. **Marketing Automation**: Generative AI can help automate marketing campaigns by generating personalized email templates, social media posts, and ad copy. It can also analyze customer data to create targeted promotions.
3. **Design and Visual Arts**: AI-powered design tools can generate images, logos, and other visual elements, freeing up designers' time for higher-level creative work.
4. **Product Development**: Generative AI can help designers and engineers develop new products by generating concepts, prototypes, and even complete product designs.
5. **Customer Service Chatbots**: AI-powered chatbots can engage with customers, answer questions, and provide support 24/7, reducing the

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [11]:
import ollama

In [ ]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries, including:

1. **Content Generation**: AI can generate high-quality content such as articles, social media posts, product descriptions, and more, freeing up human writers' time and improving efficiency.
2. **Personalized Recommendations**: Generative AI algorithms can analyze customer data and preferences to create personalized product recommendations, enhancing the shopping experience and increasing sales.
3. **Chatbots and Virtual Assistants**: AI-powered chatbots can help businesses provide 24/7 customer support, answer frequently asked questions, and route complex inquiries to human representatives.
4. **Product Design and Development**: Generative AI tools can assist designers in creating new product designs, exploring alternative configurations, and optimizing product features for improved performance and aesthetics.
5. **Marketing and Advertising**: AI-generated ad copy, images, and videos can help marke

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI

In [12]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [13]:
ed = Website("https://edwarddonner.com")
#print(ed.title)
#print(ed.text)

In [14]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [15]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
                    please provide a short summary of this website in markdown. \
                    If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [16]:
#print(user_prompt_for(ed))
wiki = Website("https://fr.wikipedia.org/wiki/Modèle_de_langage")

In [22]:
#print(user_prompt_for(wiki))

In [17]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [18]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a

In [19]:
messages_for(wiki)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Modèle de langage — Wikipédia\nThe contents of this website is as follows;                     please provide a short summary of this website in markdown.                     If it includes news or announcements, then summarize these too.\n\nAller au contenu\nMenu principal\nMenu principal\ndéplacer vers la barre latérale\nmasquer\nNavigation\nAccueil\nPortails thématiques\nArticle au hasard\nContact\nContribuer\nDébuter sur Wikipédia\nAide\nCommunauté\nModifications récentes\nRechercher\nRechercher\nApparence\nFaire un don\nCréer un compte\nSe connecter\nOutils personnels\nFaire un don\nCréer un compte\nSe connecter\nPages pour les contributeurs déconnectés\nen savoir plus\nContributions\nDiscussion\nSommaire\ndéplacer vers 

In [25]:

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

def summarize_ollama(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return print(response['message']['content'])
    

In [27]:
summarize_ollama("https://fr.wikipedia.org/wiki/Modèle_de_langage")

Le modèle de langage est un type d'intelligence artificielle conçu pour traiter et comprendre le langage naturel. Les modèles de langage sont basés sur des algorithmes d'apprentissage automatique qui utilisent des données textuelles pour apprendre à reconnaître les structures grammaticales, les relations syntaxiques et le sens du langage.

Les modèles de langage peuvent être classifiés en deux catégories principales :

1. Les modèles statiques : ces modèles sont entraînés sur une base de données textuelle fixe et ne s'adaptent pas à la fois. Ils utilisent des algorithmes tels que les réseaux neuronaux ou les arbres de décision pour analyser le langage.
2. Les modèles dynamiques : ces modèles sont entraînés en ligne, c'est-à-dire qu'ils s'adaptent à la fois sur une base de données textuelle qui change avec le temps. Ils utilisent des algorithmes tels que les réseaux neuronaux temporisés ou les modèles d'apprentissage automatique récurrents pour analyser le langage.

Les applications du 